# Goal

In this notebook, I explore the hourly 
[EIA Electric Power Operations data](https://www.eia.gov/opendata/browser/electricity/rto/region-data).

It is available in two main formats:

1. Spreadsheets (many types at different resolutions and facet hierachies)
1. [API](https://www.eia.gov/opendata/documentation.php)

**Main Question**: Are these 2 data sources consistent? (i.e. Could I train on bulk historical data downloaded via spreadsheet, then retrain periodically on recent data pulled from API?)

**Answer**: Yes, but be careful. I found that the spreadsheet version of the data has imputed/adjusted values (missing and outlier data imputed), while the API just returns the raw values (as reported by the Balancing Authorities).

Other questions:

**Q**: Can I use the API to pull all historical data? 
**A**: Yes. But you need to paginate when pulling more than 5k records.


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import requests

# Spreadsheet Data Source

This is hourly data with a lot of columns (demand, demand forecast, many generation sources, etc) and a lot of rows (every hour back to july 2015). The original spreadsheet had this data for all balancing authorities, I filtered it down to only PJM, and removed a number of columns to produce the CSV file in the `data/` directory of this project.

For each metric, it includes raw values, imputed values, and adjusted values:

- Raw: The raw metrics reported by the balancing authority.
- Imputed: Where there are missing data or outliers in the raw timeseries, EIA imputes more realistic values.
- Adjusted: A merge of the raw and imputed values. Where imputed values were required they replace the raw values.

In [2]:
# Original source: https://www.eia.gov/electricity/gridmonitor/knownissues/xls/PJM.xlsx
# Downloaded 6/3/24

df = pd.read_csv('../data/pjm_hourly_published_data_20240603.csv') 

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78840 entries, 0 to 78839
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   BA                78840 non-null  object 
 1   UTC time          78840 non-null  object 
 2   Local date        78840 non-null  object 
 3   Hour              78840 non-null  int64  
 4   Local time        78840 non-null  object 
 5   Time zone         78840 non-null  object 
 6   Generation only?  78840 non-null  object 
 7   DF                78554 non-null  object 
 8   D                 78646 non-null  object 
 9   NG                78621 non-null  object 
 10  TI                78622 non-null  object 
 11  Imputed D         175 non-null    object 
 12  Imputed NG        196 non-null    object 
 13  Imputed TI        4 non-null      float64
 14  Adjusted D        78813 non-null  object 
 15  Adjusted NG       78813 non-null  object 
 16  Adjusted TI       78622 non-null  object

In [4]:
# Data column descriptions from EIA
column_descr = {
    'BA': '2-4 letter code that identifies the balancing authority',
    'UTC time': 'The end of the hour in Coordinated Universal Time (UTC)',
    'Local date': 'The date (using local time zone) for which data has been reported',
    'Hour': 'The hour number for the day.  Hour 1 corresponds to the time period 12:00 AM - 1:00 AM',
    'Local time': 'The end of the hour in local time',
    'Time zone': 'The local time zone',
    'Generation only?': ' Y indicates the balancing authority is a generation-only BA. Generation-only BAs consist of a power plant or group of power plants and do not directly serve retail customers. Therefore, they only report net generation and interchange and do not report demand or demand forecasts.',
    'DF': 'Demand forecast (DF): Each BA produces a day-ahead electricity demand forecast for every hour of the next day. These forecasts help BAs plan for and coordinate the reliable operation of their electric system on the following day. This column displays the actual data reported to EIA in MWh.',
    'D': 'Demand (D): A calculated value representing the amount of electricity load within the balancing authority’s electric system. A BA derives its demand value by taking the total metered net electricity generation within its electric system and subtracting the total metered net electricity interchange occurring between the BA and its neighboring BAs. This column displays the actual data reported to EIA in MWh.',
    'NG': 'Net generation (NG): the metered output of electric generating units in the balancing authority’s electric system. This generation only includes generating units that are managed by the balancing authority or whose operations are visible to the balancing authority.  This column displays the actual data reported to EIA in MWh.',
    'TI': 'Total Interchange (TI): the net metered tie line flow from one BA to another directly interconnected BA. Total net interchange is the net sum of all interchange occurring between a BA and it\'s directly interconnected neighboring BAs.  Negative interchange values indicate net inflows, and positive interchange values indicate net outflows.  This column displays the actual data reported to EIA in MWh.',
    'Imputed D': 'EIA imputes for anomalous values for total demand (D) if the value is missing or reported as negative, zero, or at least 1.5 times greater than the maximum of past total demand values reported by that BA. This column displays imputed values in MWh when they are made.',
    'Imputed NG': 'EIA imputes for anomalous values for total net generation (NG) if the value is missing or reported as negative, zero, or at least 1.5 times greater than the maximum of past total net generation values reported by that BA. This column displays imputed values in MWh when they are made.',
    'Imputed TI': 'EIA imputes for anomalous values for total interchange (TI) if the value is as at least 1.5 times greater than the maximum of past positive total interchange values reported by that BA or at least 1.5 times less than the minimum of past negative total interchange values reported by that BA. This column displays imputed values in MWh when they are made.',
    'Adjusted D': 'This column displays the demand (D) reported by the balancing authority in MWh unless imputation was required. When imputation was required, this column displays the imputed demand.',
    'Adjusted NG': 'This column displays the net generation (NG) reported by the balancing authority in MWh unless imputation was required. When imputation was required, this column displays the imputed net generation.',
    'Adjusted TI': 'This column displays the total interchange (TI) reported by the balancing authority in MWh unless imputation was required. When imputation was required, this column displays the imputed total interchange.',
}

Drop columns we're ignoring for now

In [5]:
df = df[['UTC time', 'Time zone', 'DF', 'D', 'Adjusted D']]

Convert columns to appropriate types

In [6]:
df['UTC time'] = pd.to_datetime(df['UTC time'], format='%d%b%Y %H:%M:%S', utc=True)
#df['Local time'] = pd.to_datetime(df['Local time'], format='%d%b%Y %H:%M:%S').dt.tz_localize('EST')
for col in ['DF', 'D', 'Adjusted D']:
    # Handle commas in string-encoded integers
    # df.loc[:, col] = pd.to_numeric(df[col].str.replace(',', ''))
    df[col] = pd.to_numeric(df[col].str.replace(',', ''))

In [7]:
df

,UTC time,Time zone,DF,D,Adjusted D
0,2015-07-01 05:00:00+00:00,Eastern,29415.0,84024.0,84024.0
1,2015-07-01 06:00:00+00:00,Eastern,27687.0,79791.0,79791.0
2,2015-07-01 07:00:00+00:00,Eastern,26574.0,76760.0,76760.0
3,2015-07-01 08:00:00+00:00,Eastern,26029.0,74931.0,74931.0
4,2015-07-01 09:00:00+00:00,Eastern,26220.0,74368.0,74368.0
...,...,...,...,...,...
78835,2024-06-28 00:00:00+00:00,Eastern,114778.0,NaN,NaN
78836,2024-06-28 01:00:00+00:00,Eastern,110808.0,NaN,NaN
78837,2024-06-28 02:00:00+00:00,Eastern,103638.0,NaN,NaN
78838,2024-06-28 03:00:00+00:00,Eastern,95717.0,NaN,NaN


In [8]:
start = df['UTC time'].min()
end = df['UTC time'].max()

In [9]:
print(end)
pd.Timestamp('2024-06-28 04:00:00+00:00')

2024-06-28 04:00:00+00:00


Timestamp('2024-06-28 04:00:00+0000', tz='UTC')

# API Data Source

Below I query the API for all the hours covered in the spreadsheet to:

- Test out bulk querying with pagination (TLDR: works well)
- Check for consistency with the spreadsheet data (TLDR: The API gives the raw values found in the spreadsheet - not the adjusted values. So I'll need to impute missing and anamolous values myself.)

In [10]:
# Calculate the number of rows to fetch from the API between start and end
time_span = end - start
hours = int(time_span.total_seconds() / 3600) + 1

# Calculate how many paginated API requests will be required to fetch all the 
# timeseries data
# TODO: Rewrite below based on initial query to DB. B.C. hours != total records.
REQUEST_ROWS = 5000
num_full_requests = hours // REQUEST_ROWS
final_request_length = hours % REQUEST_ROWS
print(f'Fetching {hours} hours of data. Start: {start}. End: {end}')
print(f'Will make {num_full_requests} {REQUEST_ROWS}-length requests and one {final_request_length}-length request.')

Fetching 78840 hours of data. Start: 2015-07-01 05:00:00+00:00. End: 2024-06-28 04:00:00+00:00
Will make 15 5000-length requests and one 3840-length request.


In [11]:
def request_EIA_data(start_ts, end_ts, offset, length=REQUEST_ROWS):
    print(f'Fetching API page. offset:{offset}. length:{length}')
    url = ('https://api.eia.gov/v2/electricity/rto/region-data/data/?'
           'frequency=hourly&data[0]=value&facets[respondent][]=PJM&'
           'sort[0][column]=period&sort[0][direction]=asc')

    # Use list of tuples instead of dict to allow duplicate params
    params = [
      ('offset', offset),
      ('length', length),
      ('api_key', os.environ['EIA_API_KEY']),
      ('start', start_ts.strftime('%Y-%m-%dT%H')),
      ('end', end_ts.strftime('%Y-%m-%dT%H')),
      ('facets[type][]', 'D'),
      ('facets[type][]', 'DF'),
    ]

    r = requests.get(url, params=params)
    r.raise_for_status()
    return r
    

def append_EIA_page_response_df(start, end, offset, length, result_list):
    r = request_EIA_data(start, end, offset, length)
    result_df = pd.DataFrame(r.json()['response']['data'])
    print(f"  First row: {result_df.iloc[0]['period']}")
    print(f"  Last row: {result_df.iloc[-1]['period']}")
    result_list.append(result_df)

_Note_: I was assuming since there are 78840 hours, that I should fetch 78840 records. But there's up to 78839x2 records, 
because there is one record for D and a separate for DF. Also, there are some hours with neither D nor DF values - in these cases no record is returned.

Make a first request to determine the actual total number of records EIA has that match our query and time range:

In [12]:
r = request_EIA_data(start, end, 0)
r.raise_for_status()
num_total_records = int(r.json()['response']['total'])
print(f'Total records to fetch: {num_total_records}')

Fetching API page. offset:0. length:5000
Total records to fetch: 157391


In [13]:
num_full_requests = num_total_records // REQUEST_ROWS
final_request_length = num_total_records % REQUEST_ROWS

# Build a list of dataframes storing each API request (page)'s response
response_dfs = []

# Make the full-length requests
for i in range(num_full_requests):
    offset = i * REQUEST_ROWS
    append_EIA_page_response_df(start, end, offset, REQUEST_ROWS, response_dfs)
# Make the final remainder request
append_EIA_page_response_df(start, end, num_full_requests * REQUEST_ROWS, final_request_length, response_dfs)

# TODO: Note this concatenated dataframe has non-unique index since its chunked into sets of 5K
api_df = pd.concat(response_dfs)

Fetching API page. offset:0. length:5000
  First row: 2015-07-01T05
  Last row: 2015-10-13T08
Fetching API page. offset:5000. length:5000
  First row: 2015-10-13T09
  Last row: 2016-01-25T13
Fetching API page. offset:10000. length:5000
  First row: 2016-01-25T13
  Last row: 2016-05-09T06
Fetching API page. offset:15000. length:5000
  First row: 2016-05-09T06
  Last row: 2016-08-21T10
Fetching API page. offset:20000. length:5000
  First row: 2016-08-21T10
  Last row: 2016-12-03T14
Fetching API page. offset:25000. length:5000
  First row: 2016-12-03T15
  Last row: 2017-03-18T05
Fetching API page. offset:30000. length:5000
  First row: 2017-03-18T06
  Last row: 2017-06-30T09
Fetching API page. offset:35000. length:5000
  First row: 2017-06-30T10
  Last row: 2017-10-12T13
Fetching API page. offset:40000. length:5000
  First row: 2017-10-12T14
  Last row: 2018-01-25T06
Fetching API page. offset:45000. length:5000
  First row: 2018-01-25T07
  Last row: 2018-05-09T21
Fetching API page. offset

In [14]:
api_df['UTC period'] = pd.to_datetime(api_df['period'], utc=True)
api_df['value'] = pd.to_numeric(api_df['value'])

In [15]:
# Careful, EIA results can have duplicates (at the boundaries of the pages)
# And such behavior seems to be non-deterministic.
duplicate_rows = api_df[api_df.duplicated(subset=['UTC period', 'value', 'type'], keep=False)]
duplicate_rows

,period,respondent,respondent-name,type,type-name,value,value-units,UTC period
4999,2016-01-25T13,PJM,"PJM Interconnection, LLC",D,Demand,114427.0,megawatthours,2016-01-25 13:00:00+00:00
0,2016-01-25T13,PJM,"PJM Interconnection, LLC",D,Demand,114427.0,megawatthours,2016-01-25 13:00:00+00:00
4999,2016-08-21T10,PJM,"PJM Interconnection, LLC",DF,Day-ahead demand forecast,76255.0,megawatthours,2016-08-21 10:00:00+00:00
0,2016-08-21T10,PJM,"PJM Interconnection, LLC",DF,Day-ahead demand forecast,76255.0,megawatthours,2016-08-21 10:00:00+00:00
4999,2018-12-04T18,PJM,"PJM Interconnection, LLC",DF,Day-ahead demand forecast,98554.0,megawatthours,2018-12-04 18:00:00+00:00
0,2018-12-04T18,PJM,"PJM Interconnection, LLC",DF,Day-ahead demand forecast,98554.0,megawatthours,2018-12-04 18:00:00+00:00
4999,2019-10-13T17,PJM,"PJM Interconnection, LLC",D,Demand,70659.0,megawatthours,2019-10-13 17:00:00+00:00
0,2019-10-13T17,PJM,"PJM Interconnection, LLC",D,Demand,70659.0,megawatthours,2019-10-13 17:00:00+00:00
4999,2021-07-01T05,PJM,"PJM Interconnection, LLC",D,Demand,101196.0,megawatthours,2021-07-01 05:00:00+00:00
0,2021-07-01T05,PJM,"PJM Interconnection, LLC",D,Demand,101196.0,megawatthours,2021-07-01 05:00:00+00:00


In [16]:
# Remove duplicates
api_df = api_df.drop_duplicates(subset=['UTC period', 'value', 'type'])

Let's confirm that the 'Adjusted D' values in this dataset match the demand values available through the API.

Cast columns from string to appropriate types

In [17]:
len(api_df)

157385

In [18]:
api_df.head()

,period,respondent,respondent-name,type,type-name,value,value-units,UTC period
0,2015-07-01T05,PJM,"PJM Interconnection, LLC",D,Demand,84024.0,megawatthours,2015-07-01 05:00:00+00:00
1,2015-07-01T05,PJM,"PJM Interconnection, LLC",DF,Day-ahead demand forecast,29415.0,megawatthours,2015-07-01 05:00:00+00:00
2,2015-07-01T06,PJM,"PJM Interconnection, LLC",D,Demand,79791.0,megawatthours,2015-07-01 06:00:00+00:00
3,2015-07-01T06,PJM,"PJM Interconnection, LLC",DF,Day-ahead demand forecast,27687.0,megawatthours,2015-07-01 06:00:00+00:00
4,2015-07-01T07,PJM,"PJM Interconnection, LLC",DF,Day-ahead demand forecast,26574.0,megawatthours,2015-07-01 07:00:00+00:00


In [19]:
api_demand_df = api_df[api_df['type'] == 'D']

Merge spreadsheet and API values.

In [20]:
merged_df = pd.merge(df[['UTC time', 'D', 'Adjusted D']], api_demand_df[['UTC period', 'value']], 
                     left_on='UTC time', right_on='UTC period', how='inner')

merged_df.head()

,UTC time,D,Adjusted D,UTC period,value
0,2015-07-01 05:00:00+00:00,84024.0,84024.0,2015-07-01 05:00:00+00:00,84024.0
1,2015-07-01 06:00:00+00:00,79791.0,79791.0,2015-07-01 06:00:00+00:00,79791.0
2,2015-07-01 07:00:00+00:00,76760.0,76760.0,2015-07-01 07:00:00+00:00,76760.0
3,2015-07-01 08:00:00+00:00,74931.0,74931.0,2015-07-01 08:00:00+00:00,74931.0
4,2015-07-01 09:00:00+00:00,74368.0,74368.0,2015-07-01 09:00:00+00:00,74368.0


Null values are imputed in the spreadsheet's `Adjusted D` column (except in rare cases)

In [21]:
num_non_imputed = len(merged_df[merged_df['Adjusted D'].isnull() & merged_df.value.isnull()])
num_null_raw = len(merged_df[merged_df.value.isnull()])
print(f'{num_non_imputed} out of {num_null_raw} null raw values were not imputed')

0 out of 167 null raw values were not imputed


But they are not removed from the raw values of the `D` column, which matches the demand values returned by the API.

The API reported values match the spreadsheet-reported raw values, except in the last 24 hours of the spreadsheet data, there is no `Adjusted D` or `D` values. There is only a 'DF' value for these rows.

In [22]:
equal_mask = merged_df['D'] == merged_df['value']
both_nan_mask = pd.isna(merged_df['D']) & pd.isna(merged_df['value'])
equal_or_nan_mask = equal_mask | both_nan_mask
assert len(merged_df[~equal_or_nan_mask]) == 24
merged_df[~equal_or_nan_mask]

,UTC time,D,Adjusted D,UTC period,value
78810,2024-06-27 05:00:00+00:00,NaN,NaN,2024-06-27 05:00:00+00:00,98238.0
78811,2024-06-27 06:00:00+00:00,NaN,NaN,2024-06-27 06:00:00+00:00,93602.0
78812,2024-06-27 07:00:00+00:00,NaN,NaN,2024-06-27 07:00:00+00:00,89782.0
78813,2024-06-27 08:00:00+00:00,NaN,NaN,2024-06-27 08:00:00+00:00,87401.0
78814,2024-06-27 09:00:00+00:00,NaN,NaN,2024-06-27 09:00:00+00:00,86649.0
78815,2024-06-27 10:00:00+00:00,NaN,NaN,2024-06-27 10:00:00+00:00,88327.0
78816,2024-06-27 11:00:00+00:00,NaN,NaN,2024-06-27 11:00:00+00:00,91460.0
78817,2024-06-27 12:00:00+00:00,NaN,NaN,2024-06-27 12:00:00+00:00,95491.0
78818,2024-06-27 13:00:00+00:00,NaN,NaN,2024-06-27 13:00:00+00:00,98194.0
78819,2024-06-27 14:00:00+00:00,NaN,NaN,2024-06-27 14:00:00+00:00,100792.0


# Create separate columns for D and DF in API results.

In [23]:
dt_index = pd.date_range(start=start, end=end, freq='h')
dt_df = pd.DataFrame({'utc_ts': dt_index})
print(len(dt_index))
print(dt_index[0])
print(dt_index[-1])

78840
2015-07-01 05:00:00+00:00
2024-06-28 04:00:00+00:00


In [24]:
demand_df = api_df[api_df.type == 'D']
d_forecast_df = api_df[api_df.type == 'DF']

In [25]:
print(f'api_df length: {len(api_df)}')
print(f'dt_df length: {len(dt_df)}')
print(f'demand_df length: {len(demand_df)}')
print(f'd_forecast_df length: {len(d_forecast_df)}')

api_df length: 157385
dt_df length: 78840
demand_df length: 78834
d_forecast_df length: 78551


In [26]:
merge_df = pd.merge(
    dt_df,
    demand_df[['UTC period', 'respondent', 'value']].rename(columns={'value': 'D'}), 
    left_on='utc_ts', right_on='UTC period',
    how='left'
)

print('Length of first merge', len(merge_df))

merge_df = pd.merge(
    merge_df,
    d_forecast_df[['UTC period', 'value']].rename(columns={'value': 'DF'}), 
    left_on='utc_ts', right_on='UTC period',
    how='left'
)

merge_df = merge_df.drop(columns=['UTC period_x', 'UTC period_y'])
merge_df

Length of first merge 78840


,utc_ts,respondent,D,DF
0,2015-07-01 05:00:00+00:00,PJM,84024.0,29415.0
1,2015-07-01 06:00:00+00:00,PJM,79791.0,27687.0
2,2015-07-01 07:00:00+00:00,PJM,76760.0,26574.0
3,2015-07-01 08:00:00+00:00,PJM,74931.0,26029.0
4,2015-07-01 09:00:00+00:00,PJM,74368.0,26220.0
...,...,...,...,...
78835,2024-06-28 00:00:00+00:00,PJM,119396.0,114778.0
78836,2024-06-28 01:00:00+00:00,PJM,113878.0,110808.0
78837,2024-06-28 02:00:00+00:00,PJM,108814.0,103638.0
78838,2024-06-28 03:00:00+00:00,PJM,102580.0,95717.0


In [27]:
print(f'Total hours {hours}')

null_d_mask = merge_df.D.isnull()
null_df_mask = merge_df.DF.isnull()

both_null_mask = null_d_mask & null_df_mask
neither_null_mask = ~null_d_mask & ~null_df_mask
only_d_null = null_d_mask & ~null_df_mask
only_df_null = ~null_d_mask & null_df_mask

print(f'Rows divide into:')
print(f'  Both null: {both_null_mask.sum()}')
print(f'  Both non-null: {neither_null_mask.sum()}')
print(f'  Only null D: {only_d_null.sum()}')
print(f'  Only null DF: {only_df_null.sum()}')

assert hours == both_null_mask.sum() + neither_null_mask.sum() + only_d_null.sum() + only_df_null.sum()

Total hours 78840
Rows divide into:
  Both null: 145
  Both non-null: 78523
  Only null D: 28
  Only null DF: 144


Persist API-extracted dataframe for later use.

In [29]:
merge_df.to_csv('eia_api_pjm_demand.csv', index=False)